In [1]:
import torch
from datasets import load_dataset
import numpy as np


#定义数据集
class Dataset_train(torch.utils.data.Dataset):
    def __init__(self, split):
        self.dataset = load_dataset("csv",data_dir="./", data_files="data_lite_big_train.csv", split=split)
        # self.dataset = load_dataset("csv",
        # data_dir="C:/Users/19148/Documents/Pycharm_projects/paper_project", data_files="data_lite.csv", split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']

        return text, label

class Dataset_test(torch.utils.data.Dataset):
    def __init__(self, split):
        self.dataset = load_dataset("csv",data_dir="./", data_files="data_lite_big_test.csv", split=split)
        # self.dataset = load_dataset("csv",
        # data_dir="C:/Users/19148/Documents/Pycharm_projects/paper_project", data_files="data_lite.csv", split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']

        return text, label

dataset_train = Dataset_train('train')
print(len(dataset_train), dataset_train[15])
dataset_test = Dataset_test('train')
print(len(dataset_test), dataset_test[15])

Using custom data configuration default
Reusing dataset csv (C:\Users\19148\.cache\huggingface\datasets\csv\default-896d6d854b3b0933\0.0.0\2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2)


2795 ('直接替换', 1)


Using custom data configuration default
Reusing dataset csv (C:\Users\19148\.cache\huggingface\datasets\csv\default-f4c04b1f5310396e\0.0.0\2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2)


869 ('求助', 3)


In [2]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [3]:
import tqdm
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=100,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset_train,
                                     batch_size=15,
                                     collate_fn=collate_fn,
                                     # shuffle=False,
                                     shuffle=True,
                                     drop_last=True)
len_loader = len(loader)
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    print("%d/%d"%(i+1,len_loader), end=' ')
    if i%8 == 7: print('\n', end='')
    # print(labels)
    # print(input_ids.shape)
    # break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

1/186 2/186 3/186 4/186 5/186 6/186 7/186 8/186 
9/186 10/186 11/186 12/186 13/186 14/186 15/186 16/186 
17/186 18/186 19/186 20/186 21/186 22/186 23/186 24/186 
25/186 26/186 27/186 28/186 29/186 30/186 31/186 32/186 
33/186 34/186 35/186 36/186 37/186 38/186 39/186 40/186 
41/186 42/186 43/186 44/186 45/186 46/186 47/186 48/186 
49/186 50/186 51/186 52/186 53/186 54/186 55/186 56/186 
57/186 58/186 59/186 60/186 61/186 62/186 63/186 64/186 
65/186 66/186 67/186 68/186 69/186 70/186 71/186 72/186 
73/186 74/186 75/186 76/186 77/186 78/186 79/186 80/186 
81/186 82/186 83/186 84/186 85/186 86/186 87/186 88/186 
89/186 90/186 91/186 92/186 93/186 94/186 95/186 96/186 
97/186 98/186 99/186 100/186 101/186 102/186 103/186 104/186 
105/186 106/186 107/186 108/186 109/186 110/186 111/186 112/186 
113/186 114/186 115/186 116/186 117/186 118/186 119/186 120/186 
121/186 122/186 123/186 124/186 125/186 126/186 127/186 128/186 
129/186 130/186 131/186 132/186 133/186 134/186 135/186 136/186 
137

(torch.Size([15, 100]),
 torch.Size([15, 100]),
 torch.Size([15, 100]),
 tensor([0, 3, 3, 0, 2, 1, 1, 0, 2, 0, 3, 1, 2, 2, 0]))

In [ ]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

print(out.last_hidden_state.shape)
# print(out.last_hidden_state[1,0])
w2v = torch.zeros(len(dataset), 768)
label = torch.zeros(len(dataset))
batch_size = 16
len_loader = len(loader)
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader): # 这样就行了
    print("%d/%d"%(i+1,len_loader),end=' ')
    if i%8 == 7:
        print('\n', end='')
    out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)
    vec = out.last_hidden_state
    for j in range(batch_size):  # batch_size=16
        w2v[batch_size * i + j] = torch.mean(vec[j], dim=0, keepdim=True)  # 取均值，而不是第一个值
        # w2v[batch_size * i + j] = vec[j,0]  # vector
        label[batch_size * i + j] = labels[j]
print(w2v.shape, label.shape)

In [ ]:
import numpy as np
import pandas as pd
w2v_list = w2v.tolist()
# l_list = label.tolist()
total = np.concatenate((w2v_list, label.reshape(len(label), 1)), axis=1)
tp = pd.DataFrame(total)
# tp.to_csv("w2v.csv", header=None, index=None)
tp.to_csv("w2v_big_mean.csv", header=None, index=None)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 300)
    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
        out = self.fc(out.last_hidden_state)
        # out = out.softmax(dim=1)
        return out
model = Model()
w2v_low_dim = torch.zeros(len(dataset), 300)
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader): # 这样就行了
    print("%d/%d"%(i+1,len_loader),end=' ')
    if i%8 == 7:
        print('\n', end='')
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
    for j in range(batch_size):  # batch_size=16
        # w2v_low_dim[batch_size * i + j] = out[j,0]  # vector
        # 取均值，而不是第一个值
        w2v_low_dim[batch_size * i + j] = torch.mean(out[j], dim=0, keepdim=True)
print(w2v_low_dim.shape)

In [ ]:
print(w2v_low_dim.shape)
fc = torch.nn.Linear(768, 300)
temp = fc(w2v)
print(temp.shape)

In [ ]:
import numpy as np
import pandas as pd
w2v_low_dim_list = w2v_low_dim.tolist()
# w2v_low_dim_list = temp.tolist()
total_low_dim = np.concatenate((w2v_low_dim_list, label.reshape(len(label), 1)), axis=1)
tq = pd.DataFrame(total_low_dim)
# tp.to_csv("w2v.csv", header=None, index=None)
tq.to_csv("w2v_big_low_dim_mean.csv", header=None, index=None)

In [ ]:
class config(object):
    '''
    配置参数
    '''
    def __init__(self):
        # 类别数
        self.num_classes = 4

        # 整体训练次数
        self.num_epoch=3
        # batch大小
        self.batch_size=128
        #每个序列最大token数
        self.pad_size=32
        #学习率
        self.learning_rate = 1e-5

        self.bert_path='bert_pretrain'
        self.tokenizer=BertTokenizer.from_pretrained(self.bert_path) #定义分词器
        self.hidden_size=768  # Bert模型 token的embedding维度 = Bert模型后接自定义分类器（单隐层全连接网络）的输入维度

        # RNN 隐含层数量
        self.rnn_hidden_size=256
        # RNN数量
        self.num_layers=256
        # dropout
        self.dropout=0.5

In [9]:
from transformers import BertModel
from sklearn.preprocessing import MinMaxScaler

def standard_1(dim3_tensor):  # 归一化至-1 - 1
    get_min = dim3_tensor.min(dim=2)[0].unsqueeze(2) #取最小值张量 索引舍弃
    get_max = dim3_tensor.max(dim=2)[0].unsqueeze(2)
    normed_tensor = ((dim3_tensor - get_min) / (get_max - get_min) - 0.5) / 2
    return normed_tensor
# 定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #加载预训练模型
        self.fc_b = torch.nn.Linear(768, 768)
        self.pretrained = BertModel.from_pretrained('bert-base-chinese')
        for param in self.pretrained.parameters():
            param.requires_grad = True # 使参数可更新
        # 在每个RNN神经元之间使用 dropout 在单个神经元内部的两个时间步长间不使用 dropout
        self.lstm = torch.nn.LSTM(768,48,2,batch_first=True,bias=True,dropout=0.5)
        # self.lstm = torch.nn.LSTM(768,100,1,batch_first=True,bias=True, bidirectional=True)
        self.dropout = torch.nn.Dropout(0.5)
        # 双向LSTM要*2 分析LSTM节点数和网络层数时，看成神经元是LSTM全连接网络
        self.fc=torch.nn.Linear(48,4) # 自定义全连接层 ，输入数（输入的最后一个维度），输出数（多分类数量），bert模型输出的最后一个维度是768，这里的输入要和bert最后的输出统一


    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = self.pretrained(input_ids=input_ids,
            attention_mask=attention_mask, token_type_ids=token_type_ids)
        encoder, pooled = out.last_hidden_state, out.pooler_output
        encoder = self.fc_b(encoder)  # 添加全连接层
        encoder = self.fc_b(encoder)  # 添加全连接层
        # print(encoder)
        # encoder = standard_1(encoder)
        # print(encoder)
        # # bert output
        # out = self.fc(encoder[:, 0])
        # # out = out.last_hidden_state
        # out = out.softmax(dim=1)
        # print(encoder.shape, pooled.shape)

        output, _ = self.lstm(encoder)  # lstm层
        # output, _ = self.lstm(pooled)  # lstm层
        output =self.dropout(output)
        # print(output.shape)
        output =output[:,-1,:]   #encoder#只要序列中最后一个token对应的输出，（因为lstm会记录前边token的信息）
        output =self.fc(output)
        output = output.softmax(dim=1)

        return output


model = Model()

out = model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids)
print(out)
print(labels)
pre = out.argmax(dim=1)
print(pre)
out.shape

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[0.1921, 0.2658, 0.2862, 0.2559],
        [0.2190, 0.2734, 0.2465, 0.2611],
        [0.2123, 0.2685, 0.2603, 0.2589],
        [0.1912, 0.3117, 0.2410, 0.2562],
        [0.2107, 0.2575, 0.2674, 0.2645],
        [0.2027, 0.2731, 0.2630, 0.2612],
        [0.1990, 0.2661, 0.2778, 0.2571],
        [0.2297, 0.2618, 0.2617, 0.2467],
        [0.2023, 0.2945, 0.2578, 0.2454],
        [0.1919, 0.2682, 0.2598, 0.2801],
        [0.2001, 0.2811, 0.2660, 0.2528],
        [0.2168, 0.2898, 0.2388, 0.2547],
        [0.2110, 0.2758, 0.2630, 0.2502]], grad_fn=<SoftmaxBackward0>)
tensor([0, 2, 2, 3, 2, 2, 3, 0, 2, 2, 0, 2, 1])
tensor([2, 1, 1, 1, 2, 1, 2, 1, 1, 3, 1, 1, 1])


torch.Size([13, 4])

In [12]:
# from transformers import BertModel
# pretrained = BertModel.from_pretrained('bert-base-chinese')
# out = pretrained(input_ids=input_ids,
#             attention_mask=attention_mask, token_type_ids=token_type_ids)
# encoder, pooled = out.last_hidden_state, out.pooler_output
# lstm = torch.nn.LSTM(768,64,64,batch_first=True,dropout=0.5,bias=True,bidirectional=False)
# output, _ = lstm(encoder)  # lstm层
# output.type
# # output.shape
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=2,
                                     collate_fn=collate_fn,
                                     shuffle=False,
                                     # shuffle=True,
                                     drop_last=True)
len_loader = len(loader)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<function Tensor.type>

In [6]:
from transformers import AdamW
loader_train = torch.utils.data.DataLoader(dataset=dataset_train,
                                     batch_size=13,
                                     collate_fn=collate_fn,
                                     # shuffle=False,
                                     shuffle=True,
                                     drop_last=True)
#训练
# optimizer = AdamW(model.parameters(), lr=5e-4)
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader_train):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 1 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 300:
        break

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 1.3795267343521118 0.23076923076923078
1 1.3397876024246216 0.5384615384615384
2 1.3900090456008911 0.3076923076923077
3 1.3097195625305176 0.6153846153846154
4 1.3615437746047974 0.23076923076923078
5 1.3342088460922241 0.46153846153846156
6 1.2644776105880737 0.6153846153846154
7 1.2953168153762817 0.38461538461538464
8 1.3555538654327393 0.46153846153846156
9 1.3034504652023315 0.46153846153846156
10 1.3382264375686646 0.46153846153846156
11 1.3957513570785522 0.23076923076923078
12 1.3147149085998535 0.23076923076923078
13 1.3067320585250854 0.46153846153846156
14 1.3580595254898071 0.38461538461538464
15 1.350270390510559 0.38461538461538464
16 1.3052629232406616 0.38461538461538464
17 1.3296529054641724 0.3076923076923077
18 1.3284225463867188 0.3076923076923077
19 1.2896473407745361 0.6153846153846154
20 1.3403525352478027 0.46153846153846156
21 1.2944469451904297 0.46153846153846156
22 1.2755013704299927 0.5384615384615384
23 1.3134956359863281 0.46153846153846156
24 1.398621

In [7]:
#测试
from sklearn.metrics import classification_report
def test():
    model.eval()
    correct = 0
    total = 0
    label = torch.zeros(len(dataset_test))
    pre_label = torch.zeros(len(dataset_test))
    batch_size=11
    loader_test = torch.utils.data.DataLoader(dataset=dataset_test,
                                              batch_size=11,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        # if i == 5:
        #     break


        print(i,end=' ')
        if (i+1)%10 == 0 :
            print('\n',end='')

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        # print(out)
        for j in range(batch_size):  # batch_size=16
            label[batch_size * i + j] = labels[j]
            pre_label[batch_size * i + j] = out[j]
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total, correct, total)
    print(classification_report(label,pre_label,digits=4))


test()

0 1 2 3 4 5 6 7 8 9 
10 11 12 13 14 15 16 17 18 19 
20 21 22 23 24 25 26 27 28 29 
30 31 32 33 34 35 36 37 38 39 
40 41 42 43 44 45 46 47 48 49 
50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 
70 71 72 73 74 75 76 77 78 0.5788262370540852 503 869
              precision    recall  f1-score   support

         0.0     0.6667    0.7234    0.6939       235
         1.0     0.6543    0.6625    0.6584       160
         2.0     0.4963    0.7625    0.6012       261
         3.0     0.5490    0.1315    0.2121       213

    accuracy                         0.5788       869
   macro avg     0.5916    0.5700    0.5414       869
weighted avg     0.5844    0.5788    0.5414       869

